#### Do all the imports

In [1]:
import requests 
from postgres import Postgres
import dotenv
import os
from json import dump

#### load the environments

In [2]:
dotenv.load_dotenv('techtalk_SEC.secrets')
dotenv.load_dotenv('techtalk_SEC.env')

True

#### prepare  the request

In [3]:


SEC_headers= {
    "User-Agent":"PWTTechTalkBot/1.0 Port Wallis Technologies contact - rwolfe@portwallistechnologies.com",
    "Accept":"application/json"
} 

the_url = f'{os.getenv("WEB_URL")}/files/company_tickers.json'

r= requests.Request(method='GET',url=the_url,headers=SEC_headers)
s = requests.Session()

#### Go get the data

In [4]:

req= s.send( r.prepare())

#### Save the response to an external file (completely optional)

In [5]:
with open(file='cik_tickers.json',mode='w') as file:
    dump(req.json(),file)   

#### Stash the response data  in a variable 

In [6]:
the_json = req.json()

#### Stash the response in a database table

1. create the connection
1. clear out the table
1. insert the new data 

In [8]:
db = Postgres()
    
# always clear out the table first

db.run(f'truncate table "{os.getenv("PG_SCHEMA")}".company_ticker')

# bad for us to just toss away this but ...

a = [ db.run(  f"""insert  into "{os.getenv("PG_SCHEMA")}".company_ticker(
                cik_str,
                ticker,
                title) 
               values (
                   '{the_json[a]["cik_str"]}',
                   '{the_json[a]["ticker"]}',
                   '{the_json[a]["title"].replace("'","''")}') """ ) 
       for a in the_json]

#### Pull your favorite company's CIK from the database table

In [9]:
ticker_symbol = 'AES'

company_cik :str = db.one(f"""select cik_str from 
                          "{os.getenv("PG_SCHEMA")}".company_ticker where ticker = '{ticker_symbol}' """)

# CIK has to be 0 padded to 10 digits with leading 0's or it just honks 

company_cik = company_cik.rjust(10,'0')

#### And go get the list of recent submissions for that company

In [10]:

the_url = f"""{os.getenv("DATA_URL")}/submissions/CIK{company_cik}.json"""

r.url = the_url

req= s.send( r.prepare())

#### Save the results to a file again

In [11]:
with open(file=f'submissions-{ticker_symbol}.json',mode='w') as file:
    dump(req.json(),file)

#### save the submissions to a couple of  dicts for later use

In [37]:

submissions:dict = req.json()

recent_filings:dict = submissions["filings"]["recent"]


In [59]:
submission_count =  len(recent_filings["accessionNumber"])
sub_list = []
for i in range(submission_count):
    sub_list.append( {k: recent_filings[k][i] for k in recent_filings.keys()})

form10Q = [i for i in range(len(sub_list)) if sub_list[i]["form"] == "10-Q"]
latest_10Q = sub_list[form10Q[0]]
latest_10Q_1 =  {k: recent_filings[k][form10Q[0]] for k in recent_filings.keys()}
all_10Q = [] 

for i in range(len(form10Q) all_10Q = {k: recent_filings[k][form10Q[i]] for i in range(len(form10Q)) for k in recent_filings.keys()}

IndexError: list index out of range